In [13]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# display plots in this notebook
%matplotlib inline

# for store the results
import pickle
import gzip

# for include pycode
import sys
import os
sys.path.insert(0,'../pycode')

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
caffe_root = '/opt/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
from caffe import layers as L
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.


import aestheticNet

# Generacion de train y test
Primero creamos dos txt con la informacion del fichero y de su clase

In [15]:
data = pickle.load(gzip.open('../packages/info.pklz','rb',2))

In [16]:
num_images = len(data)

votesList=np.array(data.iloc[:,2:12])
auxTotal=np.sum(votesList,axis=1)
auxMeanVector=np.sum(votesList*range(1,11),axis=1)/auxTotal.astype(np.float)
auxClass=np.array(auxMeanVector >= 5, dtype=np.int)

data.loc[:,'VotesMean'] = pd.Series(auxMeanVector, index=data.index)
data.loc[:,'Class'] = pd.Series(auxClass, index=data.index)

In [18]:
data.loc[:,'id'] = data['id'].apply(str)
classes = np.array(data.sort_values(['id']).loc[:,'Class'])
ids = np.array(data.sort_values(['id']).loc[:,'id'])
aux_list =  np.array(['/home/frubio/AVA/AVADataset/{:}.jpg'.format(i) for i in ids])

In [20]:
np.random.seed(1000)
random_idx = np.random.randint(2, size=len(ids))

In [21]:
train_files = aux_list[random_idx == 0]
train_classes = classes[random_idx == 0]
test_files = aux_list[random_idx == 1]
test_classes = classes[random_idx == 1]

In [22]:
df_train = pd.DataFrame(list(zip(train_files,train_classes)))
df_train.columns = ['image_filename', 'label']
df_train.to_csv('../../AVA/train_partition_finetuning.txt', sep=' ', header=None, index=None)

df_test = pd.DataFrame(list(zip(test_files,test_classes)))
df_test.columns = ['image_filename', 'label']
df_test.to_csv('../../AVA/test_partition_finetuning.txt', sep=' ', header=None, index=None)

# Aqui comienza la chicha

In [23]:
caffe.set_device(0)  # if we have multiple GPUs, pick the first one
#caffe.set_mode_cpu()

In [24]:
weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

In [25]:
dummy_data = L.DummyData(shape=dict(dim=[1, 3, 227, 227]))
imagenet_net_filename = aestheticNet.caffenet(data=dummy_data, train=False)
imagenet_net = caffe.Net(imagenet_net_filename, weights, caffe.TEST)

In [28]:
niter = 200  # number of iterations to train

# Reset style_solver as before.
aest_solver_filename = aestheticNet.solver(aestheticNet.aest_net(train=True))
aest_solver = caffe.get_solver(aest_solver_filename)
aest_solver.net.copy_from(weights)


print ('Running solvers for %d iterations...' % niter)
solvers = [('pretrained', aest_solver)]
loss, acc, weights = aestheticNet.run_solvers(niter, solvers)
print ('Done.')

train_loss = loss['pretrained']
train_acc = acc['pretrained']
aest_weights = weights['pretrained']

# Delete solvers to save memory.
del aest_solver, solvers

Running solvers for 200 iterations...
  0) pretrained: loss=0.693, acc=72%
 10) pretrained: loss=0.785, acc=64%
 20) pretrained: loss=0.889, acc=56%
 30) pretrained: loss=1.304, acc=54%
 40) pretrained: loss=0.749, acc=68%
 50) pretrained: loss=0.673, acc=66%
 60) pretrained: loss=0.749, acc=72%
 70) pretrained: loss=1.064, acc=54%
 80) pretrained: loss=0.825, acc=66%
 90) pretrained: loss=0.893, acc=64%
100) pretrained: loss=1.155, acc=60%
110) pretrained: loss=1.541, acc=46%
120) pretrained: loss=1.221, acc=54%
130) pretrained: loss=1.181, acc=56%
140) pretrained: loss=1.511, acc=62%
150) pretrained: loss=0.685, acc=70%
160) pretrained: loss=1.848, acc=38%
170) pretrained: loss=1.563, acc=56%
180) pretrained: loss=1.017, acc=56%
190) pretrained: loss=1.337, acc=56%
199) pretrained: loss=0.957, acc=70%
Done.


In [31]:
test_net, accuracy = aestheticNet.eval_aest_net(aest_weights)
print ('Accuracy, trained from ImageNet initialization: %3.1f%%' % (100*accuracy, ))

Accuracy, trained from ImageNet initialization: 62.6%


In [36]:
test_net.blobs['fc8_aesthetic'].data

array([[-0.39917552,  0.39917469],
       [-4.64230108,  4.64230108],
       [-1.30723464,  1.30723369],
       [-2.5332756 ,  2.53327417],
       [-1.82000899,  1.82000852],
       [-1.94184339,  1.94184279],
       [-3.18653512,  3.18653321],
       [-3.02499008,  3.02498913],
       [-3.28335667,  3.28335619],
       [-4.09701347,  4.09701347],
       [-2.08524251,  2.08524251],
       [-1.13544047,  1.13543928],
       [ 0.10276354, -0.10276448],
       [-1.94203353,  1.94203234],
       [-0.64010638,  0.64010543],
       [-1.92276156,  1.92276073],
       [-3.05125403,  3.05125213],
       [-2.0735569 ,  2.07355571],
       [-0.92093992,  0.9209401 ],
       [-0.20936368,  0.20936294],
       [-0.93388695,  0.93388683],
       [-1.10290885,  1.10290718],
       [-2.44362593,  2.44362497],
       [-0.07105487,  0.07105384],
       [-2.25691867,  2.25691772],
       [-1.06756175,  1.06756115],
       [-0.85353696,  0.8535375 ],
       [-1.91471171,  1.91471052],
       [-2.78103471,